# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

##### В нашем распоряжении есть данные о нефтедобычи из трех регионов. Нам нужно оценить, какой из них будет самым перспективным

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

##### Подготовим данные, разделив на обучающую и валидационную выборки

In [3]:
features1 = df1.drop(['product', 'id'], axis = 1)
target1 = df1['product']

features2 = df2.drop(['product', 'id'], axis = 1)
target2 = df2['product']

features3 = df3.drop(['product', 'id'], axis = 1)
target3 = df3['product']

In [4]:
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features1, target1,
                                                                                  test_size = 0.25, random_state = 12345)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(features2, target2,
                                                                                  test_size = 0.25, random_state = 12345)
features_train3, features_valid3, target_train3, target_valid3 = train_test_split(features3, target3,
                                                                                  test_size = 0.25, random_state = 12345)

In [5]:
print(features_train1.shape, features_valid1.shape)
print(target_train1.shape, target_valid1.shape)

(75000, 3) (25000, 3)
(75000,) (25000,)


##### Размеры выборок в порядке. Приступим к работе с моделями

##### Первый регион:

In [6]:
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predictions1 = model1.predict(features_valid1)
print('RMSE модели:', mean_squared_error(target_valid1, predictions1) ** 0.5)
print('Средний запас предсказанного сырья:', predictions1.mean())

RMSE модели: 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438038


##### Второй регион:

In [7]:
model2 = LinearRegression()
model2.fit(features_train2, target_train2)
predictions2 = model2.predict(features_valid2)
print('RMSE модели:', mean_squared_error(target_valid2, predictions2) ** 0.5)
print('Средний запас предсказанного сырья:', predictions2.mean())

RMSE модели: 0.893099286775616
Средний запас предсказанного сырья: 68.728546895446


##### Третий регион:

In [8]:
model3 = LinearRegression()
model3.fit(features_train3, target_train3)
predictions3 = model3.predict(features_valid3)
print('RMSE модели:', mean_squared_error(target_valid3, predictions3) ** 0.5)
print('Средний запас предсказанного сырья:', predictions3.mean())

RMSE модели: 40.02970873393434
Средний запас предсказанного сырья: 94.96504596800489


##### Планируется 200 новых скважен при бюджете в 10.000млн рублей. Посчитаем минимальный объем новой скважины чтобы не уйти в минус

In [9]:
#Бюджет на новые скважины в млн
BUDGET = 10000
#Доход от одного продукта в млн
MONEY_PER_ONE = 0.45

In [10]:
print(10000 / 200 / 0.45)

111.11111111111111


##### Итого, средний объем должен быть не меньше 111.Сравним со средними объемами в регионах

In [11]:
print('Средний объем продукта с каждой скважины в первом регионе:', df1['product'].mean())
print('Средний объем продукта с каждой скважины во втором регионе:', df2['product'].mean())
print('Средний объем продукта с каждой скважины в третьем регионе:', df3['product'].mean())

Средний объем продукта с каждой скважины в первом регионе: 92.50000000000001
Средний объем продукта с каждой скважины во втором регионе: 68.82500000000002
Средний объем продукта с каждой скважины в третьем регионе: 95.00000000000004


##### Пока первый и третий регион смотрятся привлекательнее - средний объем скважин там существенно выше чем во втором, хотя и ниже чем необходимый

##### Напишем фунцию, которая принимает предсказания модели, а возвращает прибыль от 200 крупнейших скважин(в млрд.рублей)

In [12]:
def moneyhoney(target, predictions):
    sorted_predict = predictions.sort_values(ascending = False)
    sorted_sum = target[sorted_predict.index][:200]
    result = (sorted_sum.sum() - BUDGET / MONEY_PER_ONE) * MONEY_PER_ONE
    return result

In [13]:
state = np.random.RandomState(12345)

##### Используем метод subsample и найдем интересующие нас величины

In [15]:
def samples(target, predictions):
    values = []
    for i in range(1000):
        t_subsample = target.sample(n = 500, replace = True, random_state = state)
        p_subsample = predictions[t_subsample.index]
        values.append(moneyhoney(t_subsample, p_subsample))
        
    values = pd.Series(values)
    risk = (values < 0).mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975) 
    mean = values.mean()
    
    print(f'Средняя выручка: {mean:.2f} млн. руб')
    print(f'Риск убытков: {risk:.1%}')
    print(f'95% Доверительный интервал средней выручки: [{lower:.2f}; {upper:.2f}] млн. руб.')

##### Обнулим индексы чтобы не было ошибок

In [16]:
target_valid1 = target_valid1.reset_index(drop = True)
target_valid2 = target_valid2.reset_index(drop = True)
target_valid3 = target_valid3.reset_index(drop = True)
predictions1 = pd.Series(predictions1)
predictions2 = pd.Series(predictions2)
predictions3 = pd.Series(predictions3)

In [17]:
samples(target_valid1, predictions1)

Средняя выручка: 425.37 млн. руб
Риск убытков: 6.0%
95% Доверительный интервал средней выручки: [-102.09; 947.98] млн. руб.


In [18]:
samples(target_valid2, predictions2)

Средняя выручка: 518.24 млн. руб
Риск убытков: 0.3%
95% Доверительный интервал средней выручки: [128.12; 953.61] млн. руб.


In [19]:
samples(target_valid3, predictions3)

Средняя выручка: 419.92 млн. руб
Риск убытков: 6.2%
95% Доверительный интервал средней выручки: [-115.85; 989.63] млн. руб.


### Вывод:

##### Из выбранных трех регионов второй устраивает больше всего - в нем одновременно ожидается наибольшая прибыль, и наименьшие риски